In [14]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from string import punctuation
import numpy as np
import pandas as pd
import pickle, re
from typing import List, Dict, Any
import datetime

In [15]:
STORES_DICT = {
    "the_bar": "The Bar",
    "casadabebida": "Casa da Bebida"
}

In [17]:
def get_meta_info(product: str, store: str) -> Dict:
  sku = None
  brand = None
  description = None
  availability = None
  price_currency = None
  price_amount = None
  url = None

  if all_products[product][store] is not None:
    url = all_products[product][store]
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    if store=="the_bar":
      sku = soup.find("meta", property="product:sku")
      brand = soup.find("meta", property="product:brand")
      price_currency = soup.find("meta", property="product:price:currency")
      availability = soup.find("meta", property="product:availability")
      price_amount = soup.find("meta", property="product:price:amount")

      sku = sku['content'] if sku else None
      brand = brand['content'] if brand else None
      price_currency = price_currency['content'] if price_currency else None
      availability = availability['content'] if availability else None
      price_amount = float(price_amount['content']) if price_amount else None

    elif store=="casadabebida":
      sku = soup.find(itemprop="sku").get_text()
      brand = soup.find(itemprop="brand").get_text().strip()

      description = soup.find("meta", property="og:title")
      description = description['content'] if description else None

      price_currency = soup.find(itemprop="priceCurrency").get("content")
      availability = soup.find(itemprop="availability").get("content").split("/")[-1]
      price_amount = float(soup.find(itemprop="price").get_text())

  infos = {
      "sku": sku, 
      "marca": brand,
      "descricao": description,
      "disponibilidade": availability,
      "preco_moeda": price_currency,
      "preco_valor": price_amount,
      "loja": STORES_DICT[store],
      "data": datetime.datetime.now().date().isoformat(),
      "url": url
    }
  return infos

In [18]:
all_products = {
    "singleton_12y": {
        "the_bar": "https://br.thebar.com/whisky-singleton-of-dufftown-12-anos--750ml-gre30284/p",
        "casadabebida": "https://www.casadabebida.com.br/whisky/singleton-12-anos-of-glen-ord-700-ml/",
    },
    "glenkinchie_12y": {
        "the_bar": "https://br.thebar.com/whisky-glenkichie-12-anos--750ml-gre30252/p",
        "casadabebida": "https://www.casadabebida.com.br/whisky/glenkinchie-12-anos-750-ml/"
    },
    "glenfiddich_12y": {
        "the_bar": None,
        "casadabebida": "https://www.casadabebida.com.br/whisky/single-malt/whisky-glenfiddich-12-anos/"
    },
    "talisker_10y":  {
        "the_bar": "https://br.thebar.com/whisky-talisker-10-anos-750ml-gre30290/p",
        "casadabebida": "https://www.casadabebida.com.br/whisky/talisker-10-anos-750-ml/"
    },
    "johnnie_green": {
        "the_bar": "https://br.thebar.com/whisky-johnnie-walker-green-label--750ml-gre30263/p",
        "casadabebida": "https://www.casadabebida.com.br/whisky/whisky-johnnie-walker-green-label/"
    },
    "glenlivet_15y": {
        "the_bar": None,
        "casadabebida": "https://www.casadabebida.com.br/whisky/glenlivet-15-anos-750-ml/"
    },
    "laphroaig_10y": {
        "the_bar": None,
        "casadabebida": "https://www.casadabebida.com.br/whisky/whisky-laphroaig-10-anos/"
    },
    "macallan_sherryoak_12y": {
        "the_bar": None,
        "casadabebida": "https://www.casadabebida.com.br/whisky/macallan-sherry-oak-12-anos-700-ml/"
    },
    "macallan_18y": {
        "the_bar": None,
        "casadabebida": "https://www.casadabebida.com.br/whisky/whisky-the-macallan-18-anos/"
    }
  }

In [19]:
get_meta_info("glenkinchie_12y", "casadabebida")

{'sku': '873',
 'marca': 'Glenkinchie',
 'descricao': 'Whisky Glenkinchie 12 Anos 750 ml - Single Malt',
 'disponibilidade': 'InStock',
 'preco_moeda': 'BRL',
 'preco_valor': 351.41,
 'loja': 'Casa da Bebida',
 'data': '2022-04-01',
 'url': 'https://www.casadabebida.com.br/whisky/glenkinchie-12-anos-750-ml/'}

In [20]:
data = [get_meta_info(x,'casadabebida') for x in all_products.keys()]

In [21]:
dataframe = pd.DataFrame(data=data, columns = ["marca", "descricao", "disponibilidade", "loja", "sku","preco_moeda", "preco_valor", "data", "url"])
dataframe

,marca,descricao,disponibilidade,loja,sku,preco_moeda,preco_valor,data,url
0,Singleton,Whisky Singleton 12 Anos Dufftown 700 ml,InStock,Casa da Bebida,874,BRL,189.91,2022-04-01,https://www.casadabebida.com.br/whisky/singlet...
1,Glenkinchie,Whisky Glenkinchie 12 Anos 750 ml - Single Malt,InStock,Casa da Bebida,873,BRL,351.41,2022-04-01,https://www.casadabebida.com.br/whisky/glenkin...
2,Glenfiddich,Whisky Glenfiddich 12 anos 750ml,InStock,Casa da Bebida,114,BRL,284.91,2022-04-01,https://www.casadabebida.com.br/whisky/single-...
3,Talisker,Whisky Talisker 10 anos 750 ml,InStock,Casa da Bebida,872,BRL,379.91,2022-04-01,https://www.casadabebida.com.br/whisky/taliske...
4,Johnnie Walker,Whisky Johnnie Walker Green Label 750 ml,InStock,Casa da Bebida,15,BRL,379.91,2022-04-01,https://www.casadabebida.com.br/whisky/whisky-...
5,Glenlivet,Whisky Glenlivet 15 anos 750 ml,InStock,Casa da Bebida,1139,BRL,379.91,2022-04-01,https://www.casadabebida.com.br/whisky/glenliv...
6,Laphroaig,Whisky Laphroaig 10 Anos 750 ml,InStock,Casa da Bebida,750,BRL,398.91,2022-04-01,https://www.casadabebida.com.br/whisky/whisky-...
7,Macallan,Whisky Macallan Sherry Oak 12 anos 700 ml,InStock,Casa da Bebida,2744,BRL,906.21,2022-04-01,https://www.casadabebida.com.br/whisky/macalla...
8,Macallan,Whisky The Macallan 18 anos - Double Cask 700 ml,InStock,Casa da Bebida,226,BRL,3061.76,2022-04-01,https://www.casadabebida.com.br/whisky/whisky-...


In [27]:
dataframe.sort_values(by="preco_valor", ascending=True, inplace=True)

In [31]:
dataframe=dataframe.reset_index(drop=True)

In [32]:
dataframe

,marca,descricao,disponibilidade,loja,sku,preco_moeda,preco_valor,data,url
0,Singleton,Whisky Singleton 12 Anos Dufftown 700 ml,InStock,Casa da Bebida,874,BRL,189.91,2022-04-01,https://www.casadabebida.com.br/whisky/singlet...
1,Glenfiddich,Whisky Glenfiddich 12 anos 750ml,InStock,Casa da Bebida,114,BRL,284.91,2022-04-01,https://www.casadabebida.com.br/whisky/single-...
2,Glenkinchie,Whisky Glenkinchie 12 Anos 750 ml - Single Malt,InStock,Casa da Bebida,873,BRL,351.41,2022-04-01,https://www.casadabebida.com.br/whisky/glenkin...
3,Talisker,Whisky Talisker 10 anos 750 ml,InStock,Casa da Bebida,872,BRL,379.91,2022-04-01,https://www.casadabebida.com.br/whisky/taliske...
4,Johnnie Walker,Whisky Johnnie Walker Green Label 750 ml,InStock,Casa da Bebida,15,BRL,379.91,2022-04-01,https://www.casadabebida.com.br/whisky/whisky-...
5,Glenlivet,Whisky Glenlivet 15 anos 750 ml,InStock,Casa da Bebida,1139,BRL,379.91,2022-04-01,https://www.casadabebida.com.br/whisky/glenliv...
6,Laphroaig,Whisky Laphroaig 10 Anos 750 ml,InStock,Casa da Bebida,750,BRL,398.91,2022-04-01,https://www.casadabebida.com.br/whisky/whisky-...
7,Macallan,Whisky Macallan Sherry Oak 12 anos 700 ml,InStock,Casa da Bebida,2744,BRL,906.21,2022-04-01,https://www.casadabebida.com.br/whisky/macalla...
8,Macallan,Whisky The Macallan 18 anos - Double Cask 700 ml,InStock,Casa da Bebida,226,BRL,3061.76,2022-04-01,https://www.casadabebida.com.br/whisky/whisky-...
